<a href="https://colab.research.google.com/github/frau-web/npl_short_story_generator/blob/main/Short_Story_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Short-Story Generator_v2

## 1. Loading all libraries

In [1]:
#downgrading to Pytorch 1.6 to prevent error, https://forums.fast.ai/t/attributeerror-fakeloader-object-has-no-attribute-persistent-workers/81167/5
!pip install torch==1.6.0 torchvision==0.7.0 

  Using cached https://files.pythonhosted.org/packages/5d/5e/35140615fc1f925023f489e71086a9ecc188053d263d3594237281284d82/torch-1.6.0-cp37-cp37m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/4d/b5/60d5eb61f1880707a5749fea43e0ec76f27dfe69391cdec953ab5da5e676/torchvision-0.7.0-cp37-cp37m-manylinux1_x86_64.whl
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.6.0 which is incompatible.
ERROR: fastai 2.3.0 has requirement torch<1.8,>=1.7.0, but you'll have torch 1.6.0 which is incompatible.
ERROR: fastai 2.3.0 has requirement torchvision<0.9,>=0.8, but you'll have torchvision 0.7.0 which is incompatible.
  Found existing installation: torch 1.7.1
    Uninstalling torch-1.7.1:
      Successfully uninstalled torch-1.7.1
  Found existing installation: torchvision 0.8.2
    Uninstalling torchvision-0.8.2:
      Successfully uninstalled torchvision-0.8.2


In [2]:
import pandas as pd

!pip install fastai==2.0.15
!pip install fastai2==0.0.30
!pip install fastcore==1.0.16


from fastai.text.all import *

  Using cached https://files.pythonhosted.org/packages/98/2e/d4dcc69f67b4557c8543a4c65d3e136b1929b01136b227ceb986e2596825/fastai-2.0.15-py3-none-any.whl
ERROR: fastbook 0.0.16 has requirement fastai>=2.1, but you'll have fastai 2.0.15 which is incompatible.
  Found existing installation: fastai 2.3.0
    Uninstalling fastai-2.3.0:
      Successfully uninstalled fastai-2.3.0
  Using cached https://files.pythonhosted.org/packages/99/c9/bd299caa1f1c002495bc9ffb98d31605e78a131a2ba3ba66a2682a7ab245/fastcore-1.0.16-py3-none-any.whl
ERROR: nbdev 1.1.14 has requirement fastcore>=1.3.19, but you'll have fastcore 1.0.16 which is incompatible.
ERROR: fastrelease 0.1.11 has requirement fastcore>=1.3.13, but you'll have fastcore 1.0.16 which is incompatible.
ERROR: fastbook 0.0.16 has requirement fastai>=2.1, but you'll have fastai 2.0.15 which is incompatible.
  Found existing installation: fastcore 1.3.19
    Uninstalling fastcore-1.3.19:
      Successfully uninstalled fastcore-1.3.19


In [3]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

from fastbook import *
from IPython.display import display,HTML

ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.7.1 which is incompatible.


In [4]:
!pip install tokenizers
!pip install transformers #!pip install -Uq transformers

In [5]:
from transformers import GPT2LMHeadModel, GPT2TokenizerFast

In [6]:
pretrained_weights = 'gpt2'
tokenizer = GPT2TokenizerFast.from_pretrained(pretrained_weights)
model = GPT2LMHeadModel.from_pretrained(pretrained_weights)

## 2. Loading Preprocessing Text

In [7]:
# Import the data
gist = "https://gist.githubusercontent.com/frau-web/84b7a2155e97d479db0ca91b6fc3a0db/raw/89460c734dd7ba87ae6e6b4c4ffaebfa2f8f2f81/shortstories.txt"
df = pd.read_csv(gist, sep='\t', header=None) 


In [8]:
# Split into test and train
from sklearn.model_selection import train_test_split

train_test_ratio = 0.9
#train_valid_ratio = 7/9
df_full_train, df_test = train_test_split(df, train_size = train_test_ratio, random_state = 1)
#df_train, df_valid = train_test_split(df_full_train, train_size = train_valid_ratio, random_state = 1)


In [9]:
# We gather all texts in one numpy array (since it will be easier to use this way with fastai)
all_texts = np.concatenate([df_full_train[0].values, df_test[0].values])
#all_texts

In [10]:
# To process this data to train a model, we need to build a Transform that will be applied lazily.

class TransformersTokenizer(Transform):
    def __init__(self, tokenizer): self.tokenizer = tokenizer
    def encodes(self, x): 
        toks = self.tokenizer.tokenize(x)
        return tensor(self.tokenizer.convert_tokens_to_ids(toks))
    def decodes(self, x): return TitledStr(self.tokenizer.decode(x.cpu().numpy()))



Two comments on the code above:
- in encodes we don't use the tokenizer.encode method since it does some additional preprocessing for the model after tokenizing and numericalizing (the part throwing a warning before). Here we don't need any post-processing so it's fine to skip it.
- in decodes we return a TitledStr object and not just a plain string. That's a fastai class that adds a show method to the string, which will allow us to use all the fastai show methods.



In [11]:
splits = [range_of(df_full_train), list(range(len(df_full_train), len(all_texts)))]
tls = TfmdLists(all_texts, TransformersTokenizer(tokenizer), splits=splits, dl_type=LMDataLoader)

In [12]:
tls.train[0],tls.valid[0]

(tensor([1026,  373,  355, 4692,  379, 1363,  355,  287,  262, 4675,   13, 1119,  547, 1165, 3595,  284,  423]),
 tensor([  464,  6175,  2227,   284,   307,   588,   262, 11858,    13,  1406,   339,   531,   284, 31771,    11,   366,    40,   481]))

In [13]:
show_at(tls.train, 0)

It was as cold at home as in the street. They were too poor to have


The fastai library expects the data to be assembled in a DataLoaders object (something that has a training and validation dataloader). We can get one by using the dataloaders method. We just have to specify a batch size and a sequence length. We'll train with sequences of size 256 (GPT2 used sequence length 1024, but not everyone has enough GPU RAM for that):

In [14]:
bs,sl = 4,256
dls = tls.dataloaders(bs=bs, seq_len=sl)

In [15]:
dls.show_batch(max_n=2)

,text,text_
0,"tool-shed, perhaps hidden underneath a flower-pot. He began to turnAll the gods came around to see. Then Loki came up to show his things.So Brok blew as hard as he could.happened, he came out, and climbed upon a wheelbarrow and peeped over.step--pit pat, paddle pat! pit pat, waddle pat!Nutkin began again ""Arthur O'Bower has broken his band, he comes roaring up the land!When Sif woke up and saw that her hair was gone, she cried andwill fly up to Thor's house and get the hammer.""The wolf ran off and took a short way, but Red Riding-Hood stopped toOn the third day the squirrels got up very early and went fishing; theyWhen Red Riding-Hood tapped on the door, the wolf called out, ""Who isanimals.He lost one of his shoes among the cabbages, and the other shoeThey cut at the Harpies but could not hurt them.to Owl Island to gather nuts.Just then","-shed, perhaps hidden underneath a flower-pot. He began to turnAll the gods came around to see. Then Loki came up to show his things.So Brok blew as hard as he could.happened, he came out, and climbed upon a wheelbarrow and peeped over.step--pit pat, paddle pat! pit pat, waddle pat!Nutkin began again ""Arthur O'Bower has broken his band, he comes roaring up the land!When Sif woke up and saw that her hair was gone, she cried andwill fly up to Thor's house and get the hammer.""The wolf ran off and took a short way, but Red Riding-Hood stopped toOn the third day the squirrels got up very early and went fishing; theyWhen Red Riding-Hood tapped on the door, the wolf called out, ""Who isanimals.He lost one of his shoes among the cabbages, and the other shoeThey cut at the Harpies but could not hurt them.to Owl Island to gather nuts.Just then"
1,",Peter scuttered underneath the bushes. But presently, as nothingThen the same fly came in and bit him again.kill Jason.down like a little red cherry, singing ""Riddle me, riddle me, rot-tot-tote! He said to Brok, ""Now blow as hard as you can.""How big your ears are, grandma.Her feet were bare. When she left home, she had on some big slippersThen Loki said, ""I can't stand this. I will get the apples for you.""Then Sindre put a lump of iron into the fire.Peter gave himself up for lost, and shed big tears; but his sobs wereGretchen got colder and colder.a Christmas tree. Very many candles were on the tree. It was fulltied was not far off.duckling.There on the grass was a fine large sheep. This sheep had a fleece ofAnd Sindre went out. Brok blew and blew. The same mean fly came again,By and by he grew tame and let the children","Peter scuttered underneath the bushes. But presently, as nothingThen the same fly came in and bit him again.kill Jason.down like a little red cherry, singing ""Riddle me, riddle me, rot-tot-tote! He said to Brok, ""Now blow as hard as you can.""How big your ears are, grandma.Her feet were bare. When she left home, she had on some big slippersThen Loki said, ""I can't stand this. I will get the apples for you.""Then Sindre put a lump of iron into the fire.Peter gave himself up for lost, and shed big tears; but his sobs wereGretchen got colder and colder.a Christmas tree. Very many candles were on the tree. It was fulltied was not far off.duckling.There on the grass was a fine large sheep. This sheep had a fleece ofAnd Sindre went out. Brok blew and blew. The same mean fly came again,By and by he grew tame and let the children"


### - Alternative


Another way to gather the data is to preprocess the texts once and for all and only use the transform to decode the tensors to texts:

In [16]:
all_texts_copy = all_texts.copy()

In [17]:
def tokenize(text):
    toks = tokenizer.tokenize(text)
    return tensor(tokenizer.convert_tokens_to_ids(toks))

tokenized = [tokenize(t) for t in progress_bar(all_texts_copy)]

In [18]:
class TransformersTokenizer(Transform):
    def __init__(self, tokenizer): self.tokenizer = tokenizer
    def encodes(self, x): 
        return x if isinstance(x, Tensor) else tokenize(x)
        
    def decodes(self, x): return TitledStr(self.tokenizer.decode(x.cpu().numpy()))

In [19]:
tls_copy = TfmdLists(tokenized, TransformersTokenizer(tokenizer), splits=splits, dl_type=LMDataLoader)
dls = tls.dataloaders(bs=bs, seq_len=sl)

In [20]:
dls.show_batch(max_n=2)

,text,text_
0,"blackberries for supper.Twinkleberry and six other little squirrels each carried a fat minnow; butHe saw Sif lying asleep. He said, ""I am going to cut off her hair.""The King of Scots with all his power, cannot turn Arthur of the Bower!""Mr. McGregor came up with a sieve, which he intended to pop upon theWhen it was noon the ground was all plowed.So Loki went up on the porch and cut off Sif's golden hair.Then Brok showed the hammer. He said, ""This is not a very prettyLoki said, ""You will have to, if we get the hammer back.""Gretchen thought she was sitting by a big stove. It was so bright.Still Freyja said, ""I will not go."" And she was very angry. She shookOne day they were playing in the garden.Jason pushed the bulls' heads down to the ground. Then they kicked atThis looks like the end of the story; but it","berries for supper.Twinkleberry and six other little squirrels each carried a fat minnow; butHe saw Sif lying asleep. He said, ""I am going to cut off her hair.""The King of Scots with all his power, cannot turn Arthur of the Bower!""Mr. McGregor came up with a sieve, which he intended to pop upon theWhen it was noon the ground was all plowed.So Loki went up on the porch and cut off Sif's golden hair.Then Brok showed the hammer. He said, ""This is not a very prettyLoki said, ""You will have to, if we get the hammer back.""Gretchen thought she was sitting by a big stove. It was so bright.Still Freyja said, ""I will not go."" And she was very angry. She shookOne day they were playing in the garden.Jason pushed the bulls' heads down to the ground. Then they kicked atThis looks like the end of the story; but it"
1,"cautiously, peeping round the tree--there was OldNow she thought she could look into a room. In this room was a table.In the evening he came back.They were as high as a big hill. They would come close to each other,He put his head down to the water. What did he see? He saw himself init at anything, it will hit the mark and come back to you.""They did not want Jason killed. They did not know that the princessThen she put a stick into the pot and leaves grew on it.But Peter, who was very naughty, ran straight away to Mr. McGregor'she could. He said, ""It is little Red Riding-Hood, grandma.""suit. He walked and walked until he came to Thrym's house. Thrym wasWhen Sif woke up and saw that her hair was gone, she cried andThe next day Jason had to plow with the brass bulls and plant theAnd",", peeping round the tree--there was OldNow she thought she could look into a room. In this room was a table.In the evening he came back.They were as high as a big hill. They would come close to each other,He put his head down to the water. What did he see? He saw himself init at anything, it will hit the mark and come back to you.""They did not want Jason killed. They did not know that the princessThen she put a stick into the pot and leaves grew on it.But Peter, who was very naughty, ran straight away to Mr. McGregor'she could. He said, ""It is little Red Riding-Hood, grandma.""suit. He walked and walked until he came to Thrym's house. Thrym wasWhen Sif woke up and saw that her hair was gone, she cried andThe next day Jason had to plow with the brass bulls and plant theAnd Thrym"


## 3. Training (Fine-tuning the model)

In [21]:
class DropOutput(Callback):
    def after_pred(self): self.learn.pred = self.pred[0]

In [49]:
learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), cbs=[DropOutput], metrics=Perplexity()) #.to_fp16()

In [50]:
learn.validate()

(#2) [4.648604393005371,104.43912506103516]

This lists the validation loss and metrics, perplexity of 105 is pretty bad :)

In [24]:
#learn.lr_find()

In [51]:
learn.fit_one_cycle(3, 1e-3)

epoch,train_loss,valid_loss,perplexity,time
0,1.253866,4.414801,82.665359,02:59
1,1.310469,4.673801,107.104118,03:10
2,1.211378,5.010121,149.922852,03:01


In [27]:
prompt = "\n = Dog = \n \n Once upon a time there was a magic dog."

In [28]:
prompt_ids = tokenizer.encode(prompt)
inp = tensor(prompt_ids)[None] #.cuda()
inp.shape

torch.Size([1, 18])

#### (a) Output: beams search

In [29]:
preds = learn.model.generate(inp, max_length=100, num_beams=5, temperature=1.5)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [32]:
tokenizer.decode(preds[0].cpu().numpy())

'\n = Dog = \n \n Once upon a time there was a magic dog. He was named Red Riding-Hood.Then the gods said, "You may have it, if you will give me the golden fleece."Then Sindre went to the king and said, "Mr. McGregor, you may have it, if you will give me the golden fleece."The gods said, "Yes, I will."They said, "He is a very ugly duck. He'

#### (b) Ouutput: random sampling search

In [36]:
preds = learn.model.generate(inp, do_sample = True, max_length=100, top_k=50, temperature=1.5)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [37]:
tokenizer.decode(preds[0].cpu().numpy())

'\n = Dog = \n \n Once upon a time there was a magic dog.The girl did not have time till the morning.Old Brown said, "Just give me one night."But the eagle said, "Yes…. They do."THE LITTLE THOR GOT SHOROHOOD THOR GOT SHOROOT, they called the eagle "Thor!" And Peter blew and smoke and smoke from his nostrils.Loki felt rather glad at least bit him.he did'

#### (c) Unfreezing to further improve

In [38]:
learn.unfreeze()
learn.fit_one_cycle(3, 1e-3)

epoch,train_loss,valid_loss,perplexity,time
0,2.440275,3.767107,43.254726,02:39
1,2.326565,3.945486,51.701439,02:55
2,2.092449,4.219459,67.996658,02:49
3,1.843688,4.701869,110.152855,02:47
4,1.635963,4.648604,104.439125,03:13


In [39]:
preds = learn.model.generate(inp, do_sample = True, max_length=100, top_k=50, temperature=1.5)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [40]:
tokenizer.decode(preds[0].cpu().numpy())

'\n = Dog = \n \n Once upon a time there was a magic dog. She could go anywhereand took them back to him at anything.The king glad they had gold leaves, because she was so gladThor said, "Good evening, Idun has a golden crown that may beHe sent his men in ships to take Jason, but they could not get him.One can only a king and six legs haveTo some one. He is very beautiful."At last he said,'

### Part 4: Saving model

In [46]:
path = '/content/gdrive/My Drive/NLP/'

In [48]:
learn.save(path + 'Model_2')

Path('/content/gdrive/My Drive/NLP/Model_2.pth')